In [4]:
import torch
import pandas as pd
from gridFM.datasets.powergrid import GridDataset, GridDatasetMem
from gridFM.datasets.data_normalization import *
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import to_torch_coo_tensor
from torch.utils.data import ConcatDataset
import torch.nn.functional as F
from gridFM.models.graphTransformer import GraphTransformer
from tqdm import tqdm

torch.set_printoptions(precision=10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
data_path1 = r"C:\Users\MatteoMazzonelli\Desktop\gridfm_model_evaluation\scripts\..\data\case24"
scenarios = 500
norm_method = "identity"
node_normalizer, edge_normalizer = IdentityNormalizer(), IdentityNormalizer() 
mask_ratio = 0.5
mask_dim = 6
mask_value = -1

In [6]:
dataset = GridDatasetMem(
        root=data_path1,
        scenarios=scenarios,
        norm_method=norm_method,
        node_normalizer=node_normalizer,
        edge_normalizer=edge_normalizer,
        mask_ratio=mask_ratio,
        mask_dim=mask_dim,
        mask_value=mask_value,
    )

Processing...
100%|██████████| 500/500 [00:00<00:00, 1089.95it/s]
Done!


In [7]:
train_loader = DataLoader(dataset, batch_size=16, shuffle=False)

In [8]:
model = GraphTransformer(
        input_dim=9,
        hidden_dim=64,
        output_dim=6,
        edge_dim=2,
        heads=4,
        num_layers=3,
    ).to(device)

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [11]:
for batch in tqdm(train_loader):
    batch = batch.to(device)
    output = model(batch.x, batch.edge_index, batch.edge_attr)

    mask = batch.mask 
    output_masked = output[mask]  # Predictions for masked values
    y_masked = batch.y[mask] 
    loss_RMSE = torch.sqrt(F.mse_loss(output_masked, y_masked, reduction='mean'))
    print(batch.y)

    unmasked = (batch.mask == False)  # Modify this according to your mask logic 

    # Apply the unmasked values (keep only the unmasked ones from batch.y)
    output[unmasked] = batch.y[unmasked]

    V_m = output[:, 4]  # Voltage magnitudes
    V_a = output[:, 5]  # Voltage angles in degrees

    # Convert angles to radians
    V_A_rad = V_a * (torch.pi / 180)

    # Compute the complex voltage vector V
    V = V_m * torch.exp(1j * V_A_rad)

    # Compute the conjugate of V
    V_conj = torch.conj(V)

    # Reconstruct the admittance matrix (Y_bus) as a dense matrix
    # Note: Edge attributes represent the admittance values
    edge_real = batch.edge_attr[:, 0]  # Real part of Y_bus
    edge_imag = batch.edge_attr[:, 1]  # Imaginary part of Y_bus

    # Construct the sparse admittance matrix (real and imaginary parts separately)
    Y_bus_real_sparse = to_torch_coo_tensor(batch.edge_index, edge_real, size=(batch.y.size(0), batch.y.size(0)))
    Y_bus_imag_sparse = to_torch_coo_tensor(batch.edge_index, edge_imag, size=(batch.y.size(0), batch.y.size(0)))

    # Combine real and imaginary parts to create a complex sparse matrix
    Y_bus_sparse = Y_bus_real_sparse + 1j * Y_bus_imag_sparse


    # Take the conjugate of the admittance matrix
    Y_bus_conj = torch.conj(Y_bus_sparse)
    # Compute the complex power injection S_injection
    S_injection = torch.diag(V) @ Y_bus_conj @ V_conj

    net_P = output[:, 2] - output[:, 0]
    net_Q = output[:, 3] - output[:, 1]
    S_net_power_balance = net_P + 1j*net_Q
    S_net_power_balance = S_net_power_balance / 100.0

    print(S_net_power_balance)
    raise NotImplementedError

    

    loss_power = torch.sum(torch.abs(S_net_power_balance - S_injection))  # Example: Sum of absolute errors as loss

    loss = loss_RMSE + 0.0001*loss_power

    print(f"Loss: {loss.item()}")

    # Backpropagation
    optimizer.zero_grad()  # Zero the gradients from previous steps
    loss.backward()  # Compute the gradients based on the loss
    optimizer.step()  # Update the model parameters using the optimizer

  0%|          | 0/32 [00:00<?, ?it/s]

tensor([[ 58.0081176758,  22.0000000000,  62.4000167847,  12.2278327942,
           0.9964036942,  -7.6618380547],
        [ 52.0998840332,  20.0000000000,  62.4000167847,   3.9733238220,
           0.9959319234,  -7.7468466759],
        [ 96.6801986694,  37.0000000000,   0.0000000000,   0.0000000000,
           0.9712955356,  -4.4175553322],
        ...,
        [  0.0000000000,   0.0000000000, 299.9999694824, -39.2295303345,
           1.0499997139,  12.8255167007],
        [  0.0000000000,   0.0000000000, 248.6000061035,  10.9866971970,
           1.0386049747,   3.1463537216],
        [  0.0000000000,   0.0000000000,   0.0000000000,   0.0000000000,
           1.0240141153,   0.7312557101]], device='cuda:0')
tensor([ 0.1060973555-9.7721673548e-02j,  0.6727567315-1.6283561289e-01j,
         0.0303418469-3.7000000477e-01j,  0.0094240336-1.4999999106e-01j,
        -0.3813496828-1.4000000060e-01j,  0.0363884382-2.8000000119e-01j,
        -0.6669777036+4.7172316909e-01j,  0.0155730816-4.

NotImplementedError: 

In [ ]:


# Assuming batch.x has V_M (col 0) and V_A (col 1) in degrees
V_M = batch.x[:, 0]  # Voltage magnitudes
V_A = batch.x[:, 1]  # Voltage angles in degrees

# Convert angles to radians
V_A_rad = V_A * (torch.pi / 180)

# Compute the complex voltage vector V
V = V_M * torch.exp(1j * V_A_rad)

# Compute the conjugate of V
V_conj = torch.conj(V)

# Reconstruct the admittance matrix (Y_bus) as a dense matrix
# Note: Edge attributes represent the admittance values
Y_bus_dense = to_dense_adj(batch.edge_index, edge_attr=batch.edge_attr, batch=batch.batch)

# Take the conjugate of the admittance matrix
Y_bus_conj = torch.conj(Y_bus_dense)

# Compute the complex power injection S_injection
# S_injection = diag(V) @ conj(Y_bus) @ conj(V)
S_injection = torch.einsum('bi,bij,bj->b', V, Y_bus_conj, V_conj)


In [ ]:


# Load CSV into a DataFrame
df_node = pd.read_csv(file_path_node)
df_edge = pd.read_csv(file_path_edge)

# Filter for scenario 0
df_scenario_0 = df_node[df_node['scenario'] == 0.0]

df_scenario_0 = df_scenario_0.drop(columns=['scenario','bus'])
# Convert to PyTorch tensor
tensor_data = torch.tensor(df_scenario_0.values)

In [ ]:
# Get the number of buses (assuming indices are 0-based)
num_buses = int(max(df_edge['index1'].max(), df_edge['index2'].max())) + 1

# Initialize a complex matrix with zeros
adj_matrix = torch.zeros((num_buses, num_buses), dtype=torch.cdouble)

# Fill the matrix with complex values G + jB
for _, row in df_edge.iterrows():
    i, j, G, B = int(row['index1']), int(row['index2']), row['G'], row['B']
    adj_matrix[i, j] = complex(G, B)


In [ ]:
V_M = tensor_data[:, 4]  # Fourth column
V_A = tensor_data[:, 5]  # Fifth column in degrees

# Convert VA to radians
V_A_rad = V_A * (torch.pi / 180)

# Compute complex voltage vector V
V = V_M * torch.exp(1j * V_A_rad)

# Compute conjugates
V_conj = torch.conj(V)
Y_bus_conj = torch.conj(adj_matrix)

# Compute complex power injection: S_injection = diag(V) @ conj(Y_bus) @ conj(V)
S_injection = torch.diag(V) @ Y_bus_conj @ V_conj

In [ ]:
net_P = tensor_data[:, 2] - tensor_data[:, 0]
net_Q = tensor_data[:, 3] - tensor_data[:, 1]
S_net_power_balance = net_P + 1j*net_Q
S_net_power_balance = S_net_power_balance / 100.0

In [ ]:
S_net_power_balance[5]

tensor(-0.7304725920-1.2791672130j, dtype=torch.complex128)

In [ ]:
S_injection[5]

tensor(-0.7304725917-0.2799999999j, dtype=torch.complex128)

In [ ]:
S_net_power_balance - S_injection

tensor([-6.7660369668e-10-1.7388868123e-14j,
        -4.1089263936e-10-6.8278716014e-15j,
        -8.9678708903e-10-2.6741585613e-09j,
        -1.2113158254e-09-1.0081052382e-09j,
        -3.3743563499e-10-1.7792467499e-10j,
        -3.1047497906e-10-9.9916721314e-01j,
        -2.0325755246e-09-6.9388939039e-16j,
        -4.2383974108e-09-5.5446593428e-09j,
         1.7288764864e-09-4.0299094062e-09j,
         9.8632679801e-10-1.7707414424e-09j,
         1.3437029267e-09-5.0762949400e-10j,
         1.4883418942e-09-5.3685234036e-10j,
         0.0000000000e+00-5.5511151231e-16j,
        -2.8516078387e-11+1.0547118734e-14j,
         4.8730131041e-11+7.1193051454e-14j,
        -5.4651422277e-13-4.1522341121e-14j,
        -3.2462921240e-13-2.7142732506e-12j,
        -4.8649972939e-13-2.3731884513e-14j,
        -1.7629231408e-12-1.0021206087e-11j,
        -7.1886940844e-13-7.3419048618e-12j,
        -7.1054273576e-15+2.0317081351e-14j,
        -4.4853010195e-14+5.1070259133e-15j,
         2